In [1]:
# -*- coding: utf-8 -*-
"""
Created on Tue May 14 12:16:11 2019

@author: viryl
"""

from __future__ import print_function
import os
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from datetime import datetime
from ipdb import set_trace





# 定义读取数据

In [2]:
from torch.utils.data import Dataset as BaseDataset


# 加载输入

class MyDataset(BaseDataset):
    """CamVid Dataset. Read images, apply augmentation and preprocessing transformations.

    Args:
        images_dir (str): path to images folder
        masks_dir (str): path to segmentation masks folder
        class_values (list): values of classes to extract from segmentation mask
        augmentation (albumentations.Compose): data transfromation pipeline
            (e.g. flip, scale, etc.)
        preprocessing (albumentations.Compose): data preprocessing
            (e.g. noralization, shape manipulation, etc.)

    """

    def __init__(
            self,
            images_npy,
            label_npy,

           
    ):


        self.images = images_npy
        self.labels = label_npy
        self.length = images_npy.shape[0]
        # convert str names to class values on masks



    def __getitem__(self, i):

        # read data
        image = self.images[i]

        label =  self.labels[i]

         
        
   
        
        
        
        return torch.from_numpy(image), torch.from_numpy(np.array(label))
        
           



    def __len__(self):
        return self.length

# 返回一个patch 和label


  
    # 包含patch image和相应label的元组

# 画图

In [3]:
import matplotlib.pyplot as plt





def validate(net, data_loader, set_name, classes_name):
    """
    对一批数据进行预测，返回混淆矩阵以及Accuracy
    :param net:
    :param data_loader:
    :param set_name:  eg: 'valid' 'train' 'tesst
    :param classes_name:
    :return:
    """
    net.eval()
    cls_num = len(classes_name)
    conf_mat = np.zeros([cls_num, cls_num])

    for data in data_loader:
        images, labels = data
        if torch.cuda.is_available():
            images, labels = images.cuda(), labels.cuda()
        outputs = net(images)
        outputs.detach_()
        _, predicted = torch.max(outputs.data, 1)

        # 统计混淆矩阵
        for i in range(len(labels)):
            cate_i = labels[i]
            pre_i = predicted[i]
            conf_mat[cate_i, pre_i] += 1.0

    for i in range(cls_num):
        print('class:{:<10}, total num:{:<6}, correct num:{:<5}  Recall: {:.2%} Precision: {:.2%}'.format(
            classes_name[i], np.sum(
                conf_mat[i, :]), conf_mat[i, i], conf_mat[i, i] / (1 + np.sum(conf_mat[i, :])),
            conf_mat[i, i] / (1 + np.sum(conf_mat[:, i]))))

    print('{} set Accuracy:{:.2%}'.format(
        set_name, np.trace(conf_mat) / np.sum(conf_mat)))

    return conf_mat, '{:.2}'.format(np.trace(conf_mat) / np.sum(conf_mat))


# 生成图像
def show_confMat(confusion_mat, classes, set_name, out_dir):

    # 归一化
    confusion_mat_N = confusion_mat.copy()
    for i in range(len(classes)):
        confusion_mat_N[i, :] = confusion_mat[i, :] / confusion_mat[i, :].sum()

    # 获取颜色
    # 更多颜色: http://matplotlib.org/examples/color/colormaps_reference.html
    cmap = plt.cm.get_cmap('Greys')
    plt.imshow(confusion_mat_N, cmap=cmap)
    plt.colorbar()

    # 设置文字
    xlocations = np.array(range(len(classes)))
    plt.xticks(xlocations, list(classes), rotation=60)
    plt.yticks(xlocations, list(classes))
    plt.xlabel('Predict label')
    plt.ylabel('True label')
    plt.title('Confusion_Matrix_' + set_name)

    # 打印数字
    for i in range(confusion_mat_N.shape[0]):
        for j in range(confusion_mat_N.shape[1]):
            plt.text(x=j, y=i, s=int(
                confusion_mat[i, j]), va='center', ha='center', color='red', fontsize=10)
    # 保存
    plt.savefig(os.path.join(out_dir, 'Confusion_Matrix' + set_name + '.png'))
    plt.show()
    


# 定义模型

In [4]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np

# --------------------搭建网络--------------------------------


def xZero(X):
    
    X = X.cpu()
    newX = np.zeros(
        (X.shape[0], X.shape[1], X.shape[2], X.shape[3]))
    newX[:, :, int((X.shape[2]+1)/2), int((X.shape[3]+1)/2)] = X[:,
                                                                 :, int((X.shape[2]+1)/2), int((X.shape[3]+1)/2)]
    newX = torch.from_numpy(newX).double()
    if torch.cuda.is_available():
        newX = newX.cuda()
    return newX



class SeparableConv2d(nn.Module):  # Depth wise separable conv
    def __init__(self, in_channels, out_channels, kernel_size=1, stride=1, padding=0, dilation=1, bias=False):
        # 每个input channel被自己的filters卷积操作
        super(SeparableConv2d, self).__init__()
        self.conv1 = nn.Conv2d(in_channels, in_channels, kernel_size,
                               stride, padding, dilation, groups=in_channels, bias=bias)
        self.pointwise = nn.Conv2d(
            in_channels, out_channels, 1, 1, 0, 1, 1, bias=bias)

    def forward(self, x):
        
        x = self.conv1(x)
        x = self.pointwise(x)
        return x


class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        # in_channels,out_channels,kernel_size,stride(default:1),padding(default:0)
        self.conv1 = torch.nn.Sequential(
            SeparableConv2d(33, 64, 1, 1, 0),  # 1*1卷积核
            nn.ReLU(inplace=True),
            nn.GroupNorm(64, 64)
        )
        self.conv2 = nn.Sequential(
            nn.Conv2d(33, 256, 1, 1, 0),
            nn.GroupNorm(256, 256),
            nn.ReLU(inplace=True),
            nn.Dropout(0.4),
            SeparableConv2d(256, 256, 3, 1, 1),
            nn.ReLU(inplace=True),
            nn.Dropout(0.4),
            SeparableConv2d(256, 128, 3, 1, 1),
            nn.ReLU(inplace=True),
            nn.Dropout(0.4),
            SeparableConv2d(128, 64, 3, 1, 1),
            nn.ReLU(inplace=True),
            nn.GroupNorm(64, 64)
        )

        
        
    def classifier(self, input_dim):
        return nn.Sequential(
            nn.Linear(input_dim, 2048),
            nn.ReLU(inplace=True),
            nn.Dropout(0.5),
            nn.Linear(2048, 256),
            nn.ReLU(inplace=True),
            nn.Dropout(0.2),
            nn.Linear(256, 16),
            nn.LogSoftmax(dim=1)
        )
    
    
    def forward(self, x):

        x0 = xZero(x)
        x1 = self.conv1(x0)
        x2 = self.conv2(x)
       
        x12 = torch.cat((x1, x2), 1)
        xCat = x12.view(-1, self.numFeatures(x12))  # 特征映射一维展开
        
        output = self.classifier(xCat.size()[1])(xCat)
        return output

    def numFeatures(self, x):
        size = x.size()[1:]  # 获取卷积图像的h,w,depth
        num = 1
        for s in size:
            num *= s
            # print(s)
        return num

    def init_weights(self):  # 初始化权值
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                torch.nn.init.xavier_normal_(m.weight.data)
                if m.bias is not None:
                    m.bias.data.zero_()
            elif isinstance(m, nn.GroupNorm):
                m.weight.data.fill_(1)
                m.bias.data.zero_()
            elif isinstance(m, nn.Linear):
                torch.nn.init.normal_(m.weight.data, 0, 0.01)
                m.bias.data.zero_()


# 准备数据， 划分数据集

In [5]:
from __future__ import print_function
import os
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from datetime import datetime


import sklearn
from sklearn.utils.random import sample_without_replacement




# --------------------加载数据---------------------
# Indian Pines .mat文件路径(每个文件都是一个单独的类)

PATCH = np.load('/home/liyuan/dataset/HSI/高光谱医学数据/00microscope medical image data/medical sample 1-3/patch/5/PATCH.npy')
LABEL = np.load('/home/liyuan/dataset/HSI/高光谱医学数据/00microscope medical image data/medical sample 1-3/patch/5/GND.npy')







# 设置样本数

In [6]:
idx = sample_without_replacement(PATCH.shape[0],17000)
patch = PATCH[idx]
label = LABEL[idx]
label = label.astype('int16')

In [7]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(patch, label, test_size=0.6)

# 开始训练

In [8]:
# 定义超参
EPOCH = 10
BATCH_SIZE = 480
classes_name = [str(c) for c in range(3)]  # 分类地物数量

In [9]:

from tensorboardX import  SummaryWriter

training_dataset = MyDataset(x_train, y_train)
testing_dataset = MyDataset(x_test, y_test)
# Data Loaders
train_loader = torch.utils.data.DataLoader(
    dataset=training_dataset,
    batch_size=BATCH_SIZE,
    shuffle=True
)

test_loader = torch.utils.data.DataLoader(
    dataset=testing_dataset,
    batch_size=BATCH_SIZE,
    shuffle=True
)

# 检查cuda是否可用
use_cuda = torch.cuda.is_available()
# 生成log
now_time = datetime.now()
time_str = datetime.strftime(now_time, '%m-%d_%H-%M-%S')
log_path = os.path.join(os.getcwd(), "log")
log_dir = os.path.join(log_path, time_str)
if not os.path.exists(log_dir):
    os.makedirs(log_dir)
writer = SummaryWriter(log_dir)


# ---------------------搭建网络--------------------------
cnn = Net()  # 创建CNN
cnn.init_weights()  # 初始化权值
cnn = cnn.double()

# --------------------设置损失函数和优化器----------------------
optimizer = optim.Adam(cnn.parameters())  # lr:(default: 1e-3)优化器
criterion = nn.CrossEntropyLoss()  # 损失函数
scheduler = torch.optim.lr_scheduler.StepLR(
    optimizer, step_size=EPOCH/2, gamma=0.5)  # 设置学习率下降策略

# --------------------训练------------------------------
if(use_cuda):  # 使用GPU
    cnn = cnn.cuda()
for epoch in range(EPOCH):
    loss_sigma = 0.0    # 记录一个epoch的loss之和
    correct = 0.0
    total = 0.0
    scheduler.step()  # 更新学习率

    for batch_idx, data in enumerate(train_loader):
        # 获取图片和标签
        inputs, labels = data
        if(use_cuda):
            inputs, labels = inputs.cuda(), labels.cuda()
        optimizer.zero_grad()  # 清空梯度
        cnn = cnn.train()
        set_trace()
        outputs = cnn(inputs)
        loss = criterion(outputs, labels.long())
        loss.backward()  # 反向传播
        optimizer.step()  # 更新权值

        # 统计预测信息
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += ((predicted == labels).squeeze().sum()).item()
        loss_sigma += loss.item()

        # 每 BATCH_SIZE 个 iteration 打印一次训练信息，loss为 BATCH_SIZE 个 iteration 的平均
        if batch_idx % BATCH_SIZE == BATCH_SIZE-1:
            loss_avg = loss_sigma / BATCH_SIZE
            loss_sigma = 0.0
            print("Training: Epoch[{:0>3}/{:0>3}] Iteration[{:0>3}/{:0>3}] Loss: {:.4f} Acc:{:.2%}".format(
                epoch + 1, EPOCH, batch_idx + 1, len(train_loader), loss_avg, correct / total))
            # 记录训练loss
            writer.add_scalars(
                'Loss_group', {'train_loss': loss_avg}, epoch)
            # 记录learning rate
            writer.add_scalar(
                'learning rate', scheduler.get_lr()[0], epoch)
            # 记录Accuracy
            writer.add_scalars('Accuracy_group', {
                               'train_acc': correct / total}, epoch)
    # 每个epoch，记录梯度，权值
    for name, layer in cnn.named_parameters():
        writer.add_histogram(
            name + '_grad', layer.grad.cpu().data.numpy(), epoch)
        writer.add_histogram(name + '_data', layer.cpu().data.numpy(), epoch)

    # ------------------------------------ 观察模型在验证集上的表现 ------------------------------------
    if epoch % 1 == 0:
        loss_sigma = 0.0
        cls_num = len(classes_name)
        conf_mat = np.zeros([cls_num, cls_num])  # 混淆矩阵
        cnn.eval()
        for batch_idx, data in enumerate(test_loader):
            images, labels = data
            if(use_cuda):
                images, labels = images.cuda(), labels.cuda()
            cnn = cnn.train()
            outputs = cnn(images)  # forward
            outputs.detach_()  # 不求梯度
            loss = criterion(outputs, labels.long())  # 计算loss
            loss_sigma += loss.item()

            _, predicted = torch.max(outputs.data, 1)  # 统计
            # labels = labels.data    # Variable --> tensor
            # 统计混淆矩阵
            for j in range(len(labels)):
                cate_i = labels[j]
                pre_i = predicted[j]
                conf_mat[cate_i, pre_i] += 1.0
        print('{} set Accuracy:{:.2%}'.format(
            'Valid', conf_mat.trace() / conf_mat.sum()))
        # 记录Loss, accuracy
        writer.add_scalars(
            'Loss_group', {'valid_loss': loss_sigma / len(test_loader)}, epoch)
        writer.add_scalars('Accuracy_group', {
                           'valid_acc': conf_mat.trace() / conf_mat.sum()}, epoch)
print('Finished Training')

# ----------------------- 保存模型 并且绘制混淆矩阵图 -------------------------
cnn_save_path = os.path.join(log_dir, 'net_params.pkl')
torch.save(cnn.state_dict(), cnn_save_path)

conf_mat_train, train_acc = validate(cnn, train_loader, 'train', classes_name)
conf_mat_valid, valid_acc = validate(cnn, test_loader, 'test', classes_name)

show_confMat(conf_mat_train, classes_name, 'train', log_dir)
show_confMat(conf_mat_valid, classes_name, 'valid', log_dir)


/home/iiplab/ruanjian/anaconda/envs/LiYuanTorch/lib/python3.7/site-packages/torch/optim/lr_scheduler.py:100: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule.See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  "https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate", UserWarning)


> <ipython-input-9-271bfc81cdfc>(58)<module>()
     57         set_trace()
---> 58         outputs = cnn(inputs)
     59         loss = criterion(outputs, labels.long())

ipdb>  outputs = cnn(inputs)
*** RuntimeError: Expected object of device type cuda but got device type cpu for argument #1 'self' in call to _th_addmm
ipdb> q


BdbQuit: 

In [12]:
cnn(inputs)


RuntimeError: Expected object of device type cuda but got device type cpu for argument #1 'self' in call to _th_addmm

In [54]:
import torch
from torch.autograd import Variable

from graphviz import Digraph

def make_dot(var, params=None):
    """
    画出 PyTorch 自动梯度图 autograd graph 的 Graphviz 表示.
    蓝色节点表示有梯度计算的变量Variables;
    橙色节点表示用于 torch.autograd.Function 中的 backward 的张量 Tensors.

    Args:
        var: output Variable
        params: dict of (name, Variable) to add names to node that
            require grad (TODO: make optional)
    """
    if params is not None:
        assert all(isinstance(p, Variable) for p in params.values())
        param_map = {id(v): k for k, v in params.items()}

    node_attr = dict(style='filled', shape='box', align='left',
                              fontsize='12', ranksep='0.1', height='0.2')
    dot = Digraph(node_attr=node_attr, graph_attr=dict(size="12,12"))
    seen = set()

    def size_to_str(size):
        return '(' + (', ').join(['%d' % v for v in size]) + ')'

    output_nodes = (var.grad_fn,) if not isinstance(var, tuple) else tuple(v.grad_fn for v in var)

    def add_nodes(var):
        if var not in seen:
            if torch.is_tensor(var):
                # note: this used to show .saved_tensors in pytorch0.2, but stopped
                # working as it was moved to ATen and Variable-Tensor merged
                dot.node(str(id(var)), size_to_str(var.size()), fillcolor='orange')
            elif hasattr(var, 'variable'):
                u = var.variable
                name = param_map[id(u)] if params is not None else ''
                node_name = '%s\n %s' % (name, size_to_str(u.size()))
                dot.node(str(id(var)), node_name, fillcolor='lightblue')
            elif var in output_nodes:
                dot.node(str(id(var)), str(type(var).__name__), fillcolor='darkolivegreen1')
            else:
                dot.node(str(id(var)), str(type(var).__name__))
            seen.add(var)
            if hasattr(var, 'next_functions'):
                for u in var.next_functions:
                    if u[0] is not None:
                        dot.edge(str(id(u[0])), str(id(var)))
                        add_nodes(u[0])
            if hasattr(var, 'saved_tensors'):
                for t in var.saved_tensors:
                    dot.edge(str(id(t)), str(id(var)))
                    add_nodes(t)

    # 多输出场景 multiple outputs
    if isinstance(var, tuple):
        for v in var:
            add_nodes(v.grad_fn)
    else:
        add_nodes(var.grad_fn)
    return dot

In [61]:
import torch
from torch import nn
from torchviz import make_dot
from torch.autograd import Variable


x = Variable(torch.randn(1,33,5,5).double().cuda())

vis_graph = make_dot(cnn(x), params=dict(cnn.named_parameters()))
vis_graph.view()

'Digraph.gv.pdf'